In [1]:
from datetime import date
import pandas as pd
import yaml
from sqlalchemy import create_engine

# Database connection

In [2]:
# Cargar configuración
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_fuente = config['fuente']
    config_bodega = config['bodega']

In [3]:
# Crear conexiones
url_fuente = f"postgresql://{config_fuente['user']}:{config_fuente['password']}@{config_fuente['host']}:{config_fuente['port']}/{config_fuente['dbname']}"
url_bodega = f"postgresql://{config_bodega['user']}:{config_bodega['password']}@{config_bodega['host']}:{config_bodega['port']}/{config_bodega['dbname']}"

# Crear los SQLAlchemy Engine
fuente_conn = create_engine(url_fuente)
bodega_conn = create_engine(url_bodega)

# SQL Query

In [4]:
# Consulta SQL para obtener los datos de novedades del servicio
query = """
SELECT 
    n.id AS novedad_id,
    n.fecha_novedad AS fecha_hora_novedad,
    s.cliente_id,
    n.mensajero_id,
    n.tipo_novedad_id,
    n.descripcion 
FROM 
    mensajeria_novedadesservicio n
    JOIN mensajeria_servicio s ON n.id = s.cliente_id
"""

# Extract

In [6]:
# Leer datos de la fuente y dimensiones de la bodega
df = pd.read_sql(query, fuente_conn)
dim_fecha = pd.read_sql_table('dim_fecha', bodega_conn)
dim_cliente = pd.read_sql_table('dim_cliente', bodega_conn)
dim_novedad = pd.read_sql_table('dim_novedad', bodega_conn)

# Transformations

In [7]:
# Convertir fecha_novedad a datetime y extraer componentes
df['fecha_hora_novedad'] = pd.to_datetime(df['fecha_hora_novedad']).dt.date
dim_fecha['fecha'] = pd.to_datetime(dim_fecha['fecha']).dt.date

In [9]:
# Realizar los merges con las dimensiones
hecho_novedades = df.merge(
    dim_fecha[['key_dim_fecha', 'fecha']], 
    left_on='fecha_hora_novedad', 
    right_on='fecha',
    how='left'
)

hecho_novedades = hecho_novedades.merge(
    dim_cliente[['key_dim_cliente', 'cliente_id']], 
    on='cliente_id',
    how='left'
)

hecho_novedades = hecho_novedades.merge(
    dim_novedad[['key_dim_novedad', 'novedad_id']], 
    on='novedad_id',
    how='left'
)

# Seleccionar columnas finales
columnas_finales = [
    'key_dim_fecha',
    'key_dim_cliente',
    'key_dim_novedad',
    'fecha_hora_novedad',
    'descripcion'
]

hecho_novedades = hecho_novedades[columnas_finales]


In [10]:
# Agregar fecha de carga
hecho_novedades['saved'] = date.today()

# Verifications

In [11]:
# Verificaciones
print("\nInformación del DataFrame:")
print(hecho_novedades.info())
print("\nMuestra de las novedades:")
print(hecho_novedades.head())


Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28194 entries, 0 to 28193
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   key_dim_fecha       28194 non-null  int64 
 1   key_dim_cliente     28194 non-null  int64 
 2   key_dim_novedad     28194 non-null  int64 
 3   fecha_hora_novedad  28194 non-null  object
 4   descripcion         28194 non-null  object
 5   saved               28194 non-null  object
dtypes: int64(3), object(3)
memory usage: 1.3+ MB
None

Muestra de las novedades:
   key_dim_fecha  key_dim_cliente  key_dim_novedad fecha_hora_novedad  \
0            333                6                0         2023-11-30   
1            333                6                0         2023-11-30   
2            333                6                0         2023-11-30   
3            333                6                0         2023-11-30   
4            333                6

# Load

In [12]:
# Cargar en la bodega
hecho_novedades.to_sql(
    'hecho_novedades_servicio',
    bodega_conn, 
    if_exists='replace', index_label='key_hecho_novedades_servicio'
    )
   


194

# SQL Querys to response requirements

In [13]:
#Pregunta 9: ¿Cuáles son las novedades que más se presentan durante la prestación del servicio?

'''
SELECT 
    n.descripcion AS novedad,
    COUNT(*) AS cantidad_presentaciones
FROM 
    hecho_novedades_servicio hns
    JOIN dim_novedad n ON hns.key_dim_novedad = n.key_dim_novedad 
GROUP BY 
    n.descripcion
ORDER BY 
    cantidad_presentaciones DESC
'''

'\nSELECT \n    n.descripcion AS novedad,\n    COUNT(*) AS cantidad_presentaciones\nFROM \n    hecho_novedades_servicio hns\n    JOIN dim_novedad n ON hns.key_dim_novedad = n.key_dim_novedad \nGROUP BY \n    n.descripcion\nORDER BY \n    cantidad_presentaciones DESC\n'